In [1]:
import cv2
import numpy as np
import os
import random

num_images = 500
img_size = 32
num_classes = 5
fg_ratio = 0.77
alpha = 0.5

output_dirs = {
    'rgb': 'data/rgb_images',
    'depth': 'data/depth_maps',
    'label_npy': 'data/labels'
}

for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

total_fg_width = int(round(img_size * fg_ratio))
left_margin = (img_size - total_fg_width) // 2
right_margin = img_size - total_fg_width - left_margin

def random_cell_widths(num_classes, total_width, alpha):
    proportions = np.random.dirichlet([alpha] * num_classes)
    widths = (proportions * total_width).astype(int)
    diff = total_width - np.sum(widths)
    if diff != 0:
        idx = np.random.choice(num_classes)
        widths[idx] += diff
    for i in range(num_classes):
        if widths[i] < 1:
            diff = 1 - widths[i]
            widths[i] = 1
            for j in range(num_classes):
                if j != i and widths[j] > 1:
                    subtract = min(diff, widths[j] - 1)
                    widths[j] -= subtract
                    diff -= subtract
                    if diff == 0:
                        break
    return widths

def draw_large_circle_cell_rgb(img, x_min, x_max, y_min, y_max):
    cell_width = x_max - x_min
    cell_height = y_max - y_min
    cx = (x_min + x_max) // 2 + random.randint(-1, 1)
    cy = (y_min + y_max) // 2 + random.randint(-1, 1)
    margin = int(min(cell_width, cell_height) * 0.1)
    radius = max(1, min(cell_width, cell_height) // 2 - margin)
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.circle(img, (cx, cy), radius, color, -1)

def draw_large_rectangle_cell_rgb(img, x_min, x_max, y_min, y_max):
    cell_width = x_max - x_min
    cell_height = y_max - y_min
    margin_x = max(1, int(cell_width * 0.1))
    margin_y = max(1, int(cell_height * 0.1))
    pt1 = (x_min + random.randint(0, margin_x), y_min + random.randint(0, margin_y))
    pt2 = (x_max - random.randint(0, margin_x), y_max - random.randint(0, margin_y))
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(img, pt1, pt2, color, -1)

def draw_large_triangle_cell_rgb(img, x_min, x_max, y_min, y_max):
    cell_width = x_max - x_min
    cell_height = y_max - y_min
    margin_x = max(1, int(cell_width * 0.1))
    margin_y = max(1, int(cell_height * 0.1))
    pt1 = [x_min + random.randint(0, margin_x), y_min + random.randint(0, margin_y)]
    pt2 = [x_max - random.randint(0, margin_x), y_min + random.randint(0, margin_y)]
    pt3 = [(x_min + x_max) // 2 + random.randint(-margin_x, margin_x), y_max - random.randint(0, margin_y)]
    pts = np.array([pt1, pt2, pt3])
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.fillPoly(img, [pts], color)

for i in range(num_images):
    rgb_img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    label_img = np.zeros((img_size, img_size), dtype=np.uint8)
    depth_img = np.zeros((img_size, img_size), dtype=np.uint8)
    
    cell_widths = random_cell_widths(num_classes, total_fg_width, alpha)
    x_bounds = [left_margin]
    for w in cell_widths:
        x_bounds.append(x_bounds[-1] + w)
    
    classes = list(range(1, num_classes + 1))
    random.shuffle(classes)
    
    for idx, cls in enumerate(classes):
        x_min = x_bounds[idx]
        x_max = x_bounds[idx + 1]
        y_min = 0
        y_max = img_size
        
        depth_value = random.randint(50, 255)
        shape_type = random.choice(['circle', 'rectangle', 'triangle'])
        if shape_type == 'circle':
            draw_large_circle_cell_rgb(rgb_img, x_min, x_max, y_min, y_max)
        elif shape_type == 'rectangle':
            draw_large_rectangle_cell_rgb(rgb_img, x_min, x_max, y_min, y_max)
        elif shape_type == 'triangle':
            draw_large_triangle_cell_rgb(rgb_img, x_min, x_max, y_min, y_max)
        
        label_img[y_min:y_max, x_min:x_max] = cls
        depth_img[y_min:y_max, x_min:x_max] = depth_value
    
    rgb_filename = os.path.join(output_dirs['rgb'], f"rgb_{i}.png")
    depth_filename = os.path.join(output_dirs['depth'], f"depth_{i}.png")
    cv2.imwrite(rgb_filename, rgb_img)
    cv2.imwrite(depth_filename, depth_img)
    
    npy_filename = os.path.join(output_dirs['label_npy'], f"label_{i}.npy")
    np.save(npy_filename, label_img)

print(f"Dataset generated: {num_images}. File in {os.path.abspath('data')}")


Dataset generated: 500. File in /Users/ivan/Desktop/AP4Fed/dataset/iv4N/data
